In [1]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
path = './animals/raw-img'
classes = os.listdir(path)
data = {"imgpath": [], "labels": [] }

for classe in classes:
    imagesclass = os.path.join(path, classe)
    filelist = os.listdir(imagesclass)
    for image in filelist:
       data["imgpath"].append(os.path.join(imagesclass,image))
       data["labels"].append(classe)
    
dataset = pd.DataFrame(data)


lb = LabelEncoder()
dataset['encoded_labels'] = lb.fit_transform(dataset['labels'])
dataset.sample(n=5, random_state=1)

,imgpath,labels,encoded_labels
21590,./animals/raw-img\ragno\OIP-gXkw8eyrnY19blMCGd...,ragno,8
6337,./animals/raw-img\cavallo\OIP-me7nmDgO_QwDyzj7...,cavallo,1
6032,./animals/raw-img\cavallo\OIP-iyfTWYmICwO9-0ra...,cavallo,1
22295,./animals/raw-img\ragno\OIP-llpwljKwZtTrn4PTEp...,ragno,8
20032,./animals/raw-img\ragno\OIP-1whi6SMIAAR3Ajug6S...,ragno,8


In [3]:
datasetTreino, datasetTemp = train_test_split(dataset, test_size=0.3, random_state=42)
datasetTeste, datasetValid = train_test_split(datasetTemp, test_size=0.5, random_state=42)

In [4]:
generator = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
    rescale = 1. /255,
    zoom_range = 0.2,
    horizontal_flip = True
)

batch_size = 20

#Create a bottleneck file
top_model_weights_path = 'bottleneck_fc_model.h5'

# Image dimension
img_width, img_height = 224, 224

# Split the data into three categories

# 1_Training Data
train_images = generator.flow_from_dataframe(
    dataframe=datasetTreino,
    x_col='imgpath',
    y_col='labels',
    target_size=(img_width, img_height),
    class_size='categorical',
    batch_size=batch_size,
    shuffle=False
)



# 2_Validation Data
val_images = generator.flow_from_dataframe(
    dataframe=datasetValid,
    x_col='imgpath',
    y_col='labels',
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)


# 3_Test Data
test_images = generator.flow_from_dataframe(
    dataframe=datasetTeste,
    x_col='imgpath',
    y_col='labels',
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

Found 18325 validated image filenames belonging to 10 classes.
Found 3927 validated image filenames belonging to 10 classes.
Found 3927 validated image filenames belonging to 10 classes.


In [6]:
import tensorflow_hub as hub
from tensorflow.keras import layers

resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

feature_extractor_layer = hub.KerasLayer(resnet_url,
                                           trainable=False, 
                                           name='feature_extraction_layer',
                                           input_shape=(224, 224, 3))

resnet_model = tf.keras.Sequential([
    feature_extractor_layer, 
    layers.Dense(10, activation='softmax', name='output_layer')  
  ])

resnet_model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

STEP_SIZE_TRAIN=train_images.n//train_images.batch_size

resnet_model_history = resnet_model.fit(train_images, 
                        epochs=10,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=val_images)

Epoch 1/10
916/916 [==============================] - 1261s 1s/step - loss: 0.2477 - accuracy: 0.9225 - val_loss: 0.1699 - val_accuracy: 0.9501
Epoch 2/10
916/916 [==============================] - 1185s 1s/step - loss: 0.1463 - accuracy: 0.9548 - val_loss: 0.1669 - val_accuracy: 0.9544
Epoch 3/10
916/916 [==============================] - 1181s 1s/step - loss: 0.1251 - accuracy: 0.9607 - val_loss: 0.1731 - val_accuracy: 0.9524
Epoch 4/10
916/916 [==============================] - 1199s 1s/step - loss: 0.1084 - accuracy: 0.9657 - val_loss: 0.1984 - val_accuracy: 0.9503
Epoch 5/10
916/916 [==============================] - 1192s 1s/step - loss: 0.0953 - accuracy: 0.9704 - val_loss: 0.1837 - val_accuracy: 0.9539
Epoch 6/10
916/916 [==============================] - 1199s 1s/step - loss: 0.0960 - accuracy: 0.9709 - val_loss: 0.2108 - val_accuracy: 0.9496
Epoch 7/10
916/916 [==============================] - 1186s 1s/step - loss: 0.0854 - accuracy: 0.9738 - val_loss: 0.2034 - val_accuracy:

In [33]:
STEP_SIZE_TEST=test_images.n
test_images.reset() 
pred=resnet_model.predict(test_images, steps=STEP_SIZE_TEST, verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)

 197/3927 [>.............................] - ETA: 1:07:36WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 3927 batches). You may need to use the repeat() function when building your dataset.


3927/3927 [==============================] - 215s 54ms/step


In [7]:
resnet_model.evaluate(test_images)

197/197 [==============================] - 217s 1s/step - loss: 0.1800 - accuracy: 0.9516


[0.17995303869247437, 0.9516170024871826]

In [34]:
x = np.array(test_images.labels)
print(x.shape)
print(predicted_class_indices.shape)
accuracy_score(x, predicted_class_indices)

(3927,)
(3927,)


0.9590017825311943